In [1]:
import  json, pandas as pd, numpy as np, codecs
# -*- coding:utf-8 -*-
from pprint import pprint
import itertools, re
from sklearn.preprocessing import LabelEncoder

In [51]:
from os import path
import os
import re
import codecs, json
import pandas as pd
import numpy as np

In [32]:
rootdir = '../SogouC.mini/Sample'
dirs = os.listdir(rootdir)
dirs = [path.join(rootdir,f) for f in dirs if f.startswith('C')]


In [20]:
# cut word
import jieba
def cutword_1(x):
    words = jieba.cut(x)
    return ' '.join(words)

In [136]:
train = train.fillna(-1)

In [137]:
train['tind'] = train['workExperienceList'].apply(lambda x : [x[i]['industry'] for i in range(len(x)) if x[i]['industry'] is not None and i !=1])

In [138]:
train['tdep'] = train['workExperienceList'].apply(lambda x : [x[i]['department'] for i in range(len(x)) if x[i]['department'] is not None and i !=1])

In [139]:
train['tpos'] = train['workExperienceList'].apply(lambda x : [x[i]['position_name'] for i in range(len(x)) if x[i]['position_name'] is not None and i !=1])

In [140]:
#train['total_type'] = train['workExperienceList'].apply(lambda x : [x[i]['type'] for i in range(len(x)) if x[i]['type'] is not None and i !=1])
train['major'] = train['major'].fillna(u'无')
train['gender'] = train['gender'].fillna(u'男')

In [145]:
txts = zip(train.tind.values.tolist(), train.tdep.values.tolist(), train.tpos.tolist(), train.major.tolist())

In [147]:
txts = [ x+y + z + [m] for x, y, z, m in txts]

In [150]:
labels = train['workExperienceList'].apply(lambda x : x[1]['position_name'] )


In [153]:
newdf = pd.DataFrame({'label' : labels.tolist(), 'txt': txts})

In [195]:
def getJobData():
    train_list = []
    for line in open('../data/train_clean.json', 'r'):
        train_list.append(json.loads(line))
    train = pd.DataFrame(train_list)
    train = train.fillna(-1)
    train['tind'] = train['workExperienceList'].apply(lambda x : [x[i]['industry'] for i in range(len(x)) if x[i]['industry'] is not None and i !=1])
    train['tdep'] = train['workExperienceList'].apply(lambda x : [x[i]['department'] for i in range(len(x)) if x[i]['department'] is not None and i !=1])
    train['tpos'] = train['workExperienceList'].apply(lambda x : [x[i]['position_name'] for i in range(len(x)) if x[i]['position_name'] is not None and i !=1])
    train['major'] = train['major'].fillna(u'无')
    train['gender'] = train['gender'].fillna(u'男')
    txts = zip(train.tind.values.tolist(), train.tdep.values.tolist(), train.tpos.tolist(), train.major.tolist() )
    txts = [ x+y + z + [m]  for x, y, z, m in txts]
    txts = [ ' '.join([x for x in y if x is not None and  not isinstance(x, int)])  for y in txts]
    labels = train['workExperienceList'].apply(lambda x : x[1]['position_name'] )
    traindf = pd.DataFrame({'label' : labels.tolist(), 'txt': txts})
    test_list = []
    for line in open('../data/test_clean.json', 'r'):
        test_list.append(json.loads(line))
    test = pd.DataFrame(test_list)
    test = test.fillna(-1)
    test['tind'] = test['workExperienceList'].apply(lambda x : [x[i]['industry'] for i in range(len(x)) if x[i] is not None and i !=1])
    test['tdep'] = test['workExperienceList'].apply(lambda x : [x[i]['department'] for i in range(len(x)) if x[i] is not None and i !=1])
    test['tpos'] = test['workExperienceList'].apply(lambda x : [x[i]['position_name'] for i in range(len(x)) if x[i] is not None and i !=1])
    test['major'] = test['major'].fillna(u'无')
    test['gender'] = test['gender'].fillna(u'男')
    test_txts = zip(test.tind.values.tolist(), test.tdep.values.tolist(), test.tpos.tolist(), test.major.tolist() )
    test_txts = [ x+y + z + [m]  for x, y, z, m in test_txts]
    test_txts = [ ' '.join([x for x in y if x is not None and  not isinstance(x, int)]) for y in test_txts]
    return traindf, pd.DataFrame({'txt': test_txts})
traindf, testdf = getJobData()
jobs = codecs.open('../data/pos_name.txt', 'rb', encoding = 'utf-8').readlines()
jobs = [ job.strip() for job in jobs]
traindf = traindf.loc[traindf['label'].isin(jobs)]
textraw = traindf.txt.values.tolist()
textraw = [line.encode('utf-8') for line in textraw] # 需要存为str才能被keras使用

In [205]:
for x in textraw[0].split():
    print x

计算机硬件
房地产建筑建材工程
互联网电子商务
电子商务经理
运营经理
运营专员
电子商务


In [25]:
# keras处理token
maxfeatures = 50000 # 只选择最重要的词
from keras.preprocessing.text import Tokenizer
token = Tokenizer(nb_words=maxfeatures)
token.fit_on_texts(textraw) #如果文本较大可以使用文本流
text_seq = token.texts_to_sequences(textraw)

In [48]:
len(text_seq)

100

In [27]:
y = df.label.values # 定义好标签
nb_classes = len(np.unique(y))
print(nb_classes)

10


In [30]:
from __future__ import absolute_import
from keras.optimizers import RMSprop
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.layers.recurrent  import SimpleRNN, GRU, LSTM
from keras.callbacks import EarlyStopping

ImportError: cannot import name on_gpu

In [34]:
maxlen = 600 # 定义文本最大长度
batch_size = 32 # 批次
word_dim = 100 # 词向量维度
nb_filter = 200  # 卷积核个数
filter_length = 10 # 卷积窗口大小
hidden_dims = 50  # 隐藏层神经元个数
nb_epoch = 10      # 训练迭代次数
pool_length = 50   # 池化窗口大小

In [35]:
from sklearn.cross_validation import train_test_split
train_X, test_X, train_y, test_y = train_test_split(text_seq, y , train_size=0.8, random_state=1)

In [36]:
# 转为等长矩阵，长度为maxlen
print("Pad sequences (samples x time)")
X_train = sequence.pad_sequences(train_X, maxlen=maxlen,padding='post', truncating='post')
X_test = sequence.pad_sequences(test_X, maxlen=maxlen,padding='post', truncating='post')
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Pad sequences (samples x time)
('X_train shape:', (80, 600))
('X_test shape:', (20, 600))


In [40]:

# 将y的格式展开成one-hot
Y_train = np_utils.to_categorical(train_y, nb_classes)
Y_test = np_utils.to_categorical(test_y, nb_classes)

NameError: name 'np_utils' is not defined

In [41]:
# CNN 模型
print('Build model...')
model = Sequential()

# 词向量嵌入层，输入：词典大小，词向量大小，文本长度
model.add(Embedding(maxfeatures, word_dim,input_length=maxlen)) 
model.add(Dropout(0.25))
model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode="valid",
                        activation="relu"))
# 池化层
model.add(MaxPooling1D(pool_length=pool_length))
model.add(Flatten())
# 全连接层
model.add(Dense(hidden_dims))
model.add(Dropout(0.25))
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

Build model...


NameError: name 'Sequential' is not defined

In [42]:
earlystop = EarlyStopping(monitor='val_loss', patience=1, verbose=1)
result = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, 
            validation_split=0.1, show_accuracy=True,callbacks=[earlystop])

NameError: name 'EarlyStopping' is not defined

In [43]:
score = earlystop.model.evaluate(X_test, Y_test, batch_size=batch_size)
print('Test score:', score)
classes = earlystop.model.predict_classes(X_test, batch_size=batch_size)
acc = np_utils.accuracy(classes, test_y) # 要用没有转换前的y
print('Test accuracy:', acc)

NameError: name 'earlystop' is not defined

In [44]:
# LSTM
print('Build model...')
model = Sequential()

# 词向量嵌入层，输入：词典大小，词向量大小，文本长度
model.add(Embedding(maxfeatures, word_dim,input_length=maxlen)) 
#model.add(Dropout(0.25))
model.add(LSTM(100)) 
model.add(Flatten())
# 全连接层
model.add(Dense(hidden_dims))
model.add(Dropout(0.25))
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

Build model...


NameError: name 'Sequential' is not defined

In [45]:
result = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=1, 
            validation_split=0.1, show_accuracy=True)

NameError: name 'model' is not defined

In [46]:
textseq

NameError: name 'textseq' is not defined